### RNN의 각 Step에 넣어줄 데이터 준비 - 사전 연습

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
print(tf.__version__)

1.1.0


In [2]:
sess = tf.InteractiveSession()

In [3]:
batch_size = 3
n_steps = 4
n_inputs = 5

values = []
for i in range(0, batch_size):
    values.append([])
    for j in range(0, n_steps):
        values[i].append([])
        for k in range(0, n_inputs):
            values[i][j].append(k + j + (i * 5))
            
values_array = np.asarray(values)
x = tf.constant(value = values_array)
print(x)
print(x.eval())

Tensor("Const:0", shape=(3, 4, 5), dtype=int64)
[[[ 0  1  2  3  4]
  [ 1  2  3  4  5]
  [ 2  3  4  5  6]
  [ 3  4  5  6  7]]

 [[ 5  6  7  8  9]
  [ 6  7  8  9 10]
  [ 7  8  9 10 11]
  [ 8  9 10 11 12]]

 [[10 11 12 13 14]
  [11 12 13 14 15]
  [12 13 14 15 16]
  [13 14 15 16 17]]]


In [4]:
xt = tf.transpose(x, perm = [1, 0, 2])
print(xt)
print(xt.eval())

Tensor("transpose:0", shape=(4, 3, 5), dtype=int64)
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]]

 [[ 1  2  3  4  5]
  [ 6  7  8  9 10]
  [11 12 13 14 15]]

 [[ 2  3  4  5  6]
  [ 7  8  9 10 11]
  [12 13 14 15 16]]

 [[ 3  4  5  6  7]
  [ 8  9 10 11 12]
  [13 14 15 16 17]]]


In [5]:
xr = tf.reshape(xt, [-1, n_inputs])  # n_inputs = 5
print(xr)
print(xr.eval())

Tensor("Reshape:0", shape=(12, 5), dtype=int64)
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [ 2  3  4  5  6]
 [ 7  8  9 10 11]
 [12 13 14 15 16]
 [ 3  4  5  6  7]
 [ 8  9 10 11 12]
 [13 14 15 16 17]]


In [6]:
xs = tf.split(value = xr, num_or_size_splits = n_steps, axis=0)  # n_steps = 4
print(type(xs))
print()
print(xs)
print()
for item in xs:
    print(item.eval())
    print()

<class 'list'>

[<tf.Tensor 'split:0' shape=(3, 5) dtype=int64>, <tf.Tensor 'split:1' shape=(3, 5) dtype=int64>, <tf.Tensor 'split:2' shape=(3, 5) dtype=int64>, <tf.Tensor 'split:3' shape=(3, 5) dtype=int64>]

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]

[[ 2  3  4  5  6]
 [ 7  8  9 10 11]
 [12 13 14 15 16]]

[[ 3  4  5  6  7]
 [ 8  9 10 11 12]
 [13 14 15 16 17]]



### MNIST 데이터에 대한 RNN 배치 Input 처리 - 사전연습

In [7]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
# image: 28 * 30
batch_size = 128
n_steps = 28
n_inputs = 28

x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
print(x.get_shape())
print()

xt = tf.transpose(x, perm = [1, 0, 2])
print(xt.get_shape())
print()

xr = tf.reshape(xt, shape = [-1, n_inputs])
print(xr.get_shape())
print()

xs = tf.split(value=xr, num_or_size_splits=n_steps, axis=0)
print(len(xs))
for item in xs:
    print(item)

(?, 28, 28)

(28, ?, 28)

(?, 28)

28
Tensor("split_1:0", shape=(?, 28), dtype=float32)
Tensor("split_1:1", shape=(?, 28), dtype=float32)
Tensor("split_1:2", shape=(?, 28), dtype=float32)
Tensor("split_1:3", shape=(?, 28), dtype=float32)
Tensor("split_1:4", shape=(?, 28), dtype=float32)
Tensor("split_1:5", shape=(?, 28), dtype=float32)
Tensor("split_1:6", shape=(?, 28), dtype=float32)
Tensor("split_1:7", shape=(?, 28), dtype=float32)
Tensor("split_1:8", shape=(?, 28), dtype=float32)
Tensor("split_1:9", shape=(?, 28), dtype=float32)
Tensor("split_1:10", shape=(?, 28), dtype=float32)
Tensor("split_1:11", shape=(?, 28), dtype=float32)
Tensor("split_1:12", shape=(?, 28), dtype=float32)
Tensor("split_1:13", shape=(?, 28), dtype=float32)
Tensor("split_1:14", shape=(?, 28), dtype=float32)
Tensor("split_1:15", shape=(?, 28), dtype=float32)
Tensor("split_1:16", shape=(?, 28), dtype=float32)
Tensor("split_1:17", shape=(?, 28), dtype=float32)
Tensor("split_1:18", shape=(?, 28), dtype=float32)
Ten

In [43]:
n_state_units = 20
with tf.variable_scope('rnn', reuse=True):
    rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_state_units, activation=t, reuse=True)
    print(rnn_cell.output_size) # RNN cell의 output node의 수
    print(rnn_cell.state_size) # RNN cell의 hidden node의 수
    
    initial_state = rnn_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    print(initial_state)
    
    outputs, state = tf.contrib.rnn.static_rnn(cell = rnn_cell, inputs = xs, initial_state=initial_state, dtype=tf.float32)
    print(len(outputs))
    print(outputs[-1])

NameError: name 'tanh' is not defined

In [36]:
? tf.contrib.rnn.BasicRNNCell.zero_state